# UltimateGuitarTabs Scraper

The following notebook scrapes the top 1000 most popular tabs posting on https://www.ultimate-guitar.com/

After scraping, it stores the data found into a MySQL database ('UltimateGuitarTabs') with the following tables:
1. Tab_Data (id, song, artist, is_acoustic, tab_url)
    - id: Unique tab ID 
    - song: Name of song
    - artist: Name of artist
    - is_acoustic: Song is played acoustically (1), otherwise (0)
    - tab_url: URL to tab
2. Artists (name, url)
    - name: Name of artist
    - url: URL to artist profile
3. Hits (id, num_hits, votes, rating)
    - id: Unique tab ID
    - num_hits: # of times tab visited
    - votes: # number of votes received
    - rating: Avg rating out of 5
4. Chords (song, artist, tonality, capo, chords)
    - song: Name of song
    - artist: Name of artist
    - tonality: Original of song
    - capo: Fret to place capo
    - chords: String of full chord progression
    
Scraped on June 5, 2018

In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import json
import pymysql.cursors
import pandas as pd
import time
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:20]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
def connect_to_database(db="UltimateGuitarTabs"):
    """
    Connects to MySQL database and 
    returns the connection"""
    db = pymysql.connect(host="localhost",  # your host 
                         user="root",       # username
                         passwd="",     # password
                         db=db)   # name of the database
    return db

#proxy = next(proxy_pool)
def simple_get(url):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        #with closing(get(url, stream=True, proxies={"http": proxy, "https": proxy})) as resp:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                print(resp.status_code)
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = None
    if 'Content-Type' in resp.headers:
        content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    Source: https://realpython.com/python-web-scraping-practical-introduction/
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

def get_data(url):
    """
    
    """
    global proxy
    html = ''
    soup = ''
    while html == '' or soup == '':
        try:
            html = simple_get(url)
            soup = BeautifulSoup(html, 'html.parser')
            #Data is stored as JSON on the page   
            script = soup.find('script', text=re.compile('window.UGAPP.store.page'))
            # Removes unnecessary text
            json_text = re.search(r'^\s*window\.UGAPP\.store\.page\s*=\s*({.*?})\s*;\s*$', \
                              script.string, flags=re.DOTALL | re.MULTILINE).group(1)
            data = json.loads(json_text)
            return(data)
        except Exception as e:
            print('Exception: '+str(e))
            print(url)
            #proxy = next(proxy_pool)
            #print('new proxy:',proxy)

            time.sleep(5)
            continue

def get_tabs_data(url):
    """
    Creates the bs4 object and extracts a list
    of tab info. Hits info is stored as a separate 
    list in the html file so it is returned separately.
    """
    
    data = get_data(url)
    tab_links = data['data']['data']['tabs']       
    hits = data['data']['data']['hits']
    return(tab_links, hits)

def get_chords(url):
    """
    Scrapes and returns the sequences of 
    chords as a list as well as the fret number
    to place a capo. 
    """

    data = get_data(url)

    chords = data['data']['tab_view']['wiki_tab']['content']
    
    # Matching groups (open tag)(chord pitch)(base note {0 or 1})(chord type)(base note {0 or 1})(closing tag)
    pattern = "(\[ch\])([A-G]+)(\/[A-G]*[b#])*([(?m)|(?m\d)|(?b\d)|(?#\d)|(?maj\d)|(?add\d)|(?sus\d)|(?aug)|(?aug\d)|(?dim)|(?dim\d)]*)(\/[A-G]*[b#])*(\[\/ch\])"
    prog = re.compile(pattern)
    result = prog.findall(chords)
    
    cleaned_res = result
    for i in range(len(result)):
        # Grabbing groups (chord pitch)(base note)(chord type)(base note)
        cleaned_res[i] = result[i][1] + result[i][2] + result[i][3] + result[i][4]
       
    # Grabbing Capo info
    capo = 0
    try:
        capo = data['data']['tab_view']['meta']['capo']
    except:
        capo = 0

    return(cleaned_res, capo)
    
def get_genre(url):
    """
    Grabs the artist's categorized genre
    """
    data = get_data(url)
    genre = data['data']['artist']['genre']
    
    return(genre)

def add_to_db(tab_list, hits, recreate=False):
    """
    Creates and adds data to four MySQL tables
    """
    db = connect_to_database()
    cur = db.cursor()
    
    if recreate:
        delete_tabs = "DROP TABLE IF EXISTS Tab_Data;"
        cur.execute(delete_tabs)
        delete_artists = "DROP TABLE IF EXISTS Artists;"
        cur.execute(delete_artists)
        delete_hits = "DROP TABLE IF EXISTS Hits;"
        cur.execute(delete_hits)
        delete_chords = "DROP TABLE IF EXISTS Chords;"
        cur.execute(delete_chords)


        create_tabs = "CREATE TABLE Tab_Data (id INT(11) NOT NULL PRIMARY KEY, song CHAR(50), artist CHAR(50), \
                                    is_acoustic INT(1), tab_url TEXT(500));"
        cur.execute(create_tabs)

        create_artists = "CREATE TABLE Artists (name CHAR(50) NOT NULL PRIMARY KEY, url TEXT(500));"
        cur.execute(create_artists)


        create_hits = "CREATE TABLE Hits (id INT(11) NOT NULL PRIMARY KEY, num_hits INT(8), \
                       votes INT(11), rating FLOAT);"
        cur.execute(create_hits)

        create_chords = "CREATE TABLE Chords (id INT(11) NOT NULL PRIMARY KEY, song CHAR(50), artist CHAR(50), tonality CHAR(3), capo INT(2), chords TEXT(500));"
        cur.execute(create_chords)
        seen_id = set()
    else:
        cur.execute("SELECT Tab_Data.id FROM Tab_Data JOIN Chords on Tab_Data.id = Chords.id Join Hits on Tab_Data.id = Hits.id")
        seen_id = set(id[0] for id in cur.fetchall())
            
    for i in range(len(tab_list)):
        if (i+1)%50==0:
            print(i+1)
        song = tab_list[i]    
        hit = hits[i]

        tab_id = int(song['id'])
        if tab_id in seen_id:
            continue
        seen_id.add(tab_id)
        song_name = song['song_name'][:50].replace("'","\\'")
        artist = song['artist_name'][:50].replace("'","\\'")
        tonality = song['tonality_name']
        votes = int(song['votes'])
        rating = float(song['rating'])
        is_acoustic = int(song['recording']['is_acoustic'])
        tab_url = song['tab_url']
        artist_url = song['artist_url']
        hit_id = int(hit['id'])
        hit_num = int(hit['hits'])
        chords, capo = get_chords(song['tab_url'])
        
        sql_tab = "INSERT INTO Tab_Data (id,song,artist,is_acoustic,tab_url) \
        VALUES ('%d','%s','%s','%d','%s')" % \
        (tab_id, song_name, artist, is_acoustic, tab_url)

        sql_artist = "INSERT INTO Artists (name, url) VALUES ('%s','%s')" %(artist, artist_url)
        
        sql_hit = "INSERT INTO Hits (id, num_hits, votes, rating) VALUES ('%d','%d','%d','%.8f')" % (tab_id, hit_num, votes, rating)

        sql_chords = "INSERT INTO Chords (id, song, artist, tonality, capo, chords) VALUES('%d','%s','%s','%s','%d','%s')" % \
        (tab_id, song_name, artist, tonality, int(capo), ','.join(chords))
        
        
        try:
            cur.execute(sql_tab)
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()
        
        try:
            cur.execute(sql_artist)
            db.commit()
        except Exception as e:
            #print(e)
            db.rollback()
            
        try:
            cur.execute(sql_hit)
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()

        try:
            cur.execute(sql_chords)
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()
            
    # disconnect from server
    db.close()
    print('done')
    return True

def get_multiple_pages(url, genres, years, n):
    """
    Creates functionality to scrape multiple
    pages up to n
    """
    page_suffix = "&page="
    year_suffix = "&decade[]="
    genre_suffix = "&genres[]="
    tabs_list = []
    hits_list = []
    
    for genre in genres:
        for year in range(years[0], years[1]+1, 10):
            for i in range(n):
                print(genre,year,i+1)
                cur_tabs, cur_hits = get_tabs_data(url + genre_suffix + str(genre) + year_suffix + str(year) + page_suffix + str(i))
                tabs_list += cur_tabs
                hits_list += cur_hits

    print('found %d urls'%len(tabs_list))
    return(tabs_list, hits_list)
   
def print_table(table, count=False):
    """
    Prints database table
    """
    db = connect_to_database()
    cur = db.cursor()

    # Select data from table using SQL query.
    if count:
        cur.execute("SELECT COUNT(*) FROM "+table)
        cnt = cur.fetchall()[0][0]
    else:    
        cnt = cur.execute("SELECT * FROM "+table)
        for row in cur.fetchall() :
            print(row)
    print(table, cnt)
    return(cnt)

In [ ]:
tabs, hits = get_multiple_pages("https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords", [4,14,49,666], [1960,2010], 20)

In [ ]:
add_to_db(tabs, hits, recreate=False)
print_table('tab_data',count=True)
print_table('chords', count=True)